# ローカルユーザを削除します
<HR>

ローカルユーザとして登録済みのユーザを削除します。<br/>

次のセルで、下記のように削除するユーザのメールアドレスを配列`LOCAL_USERS_DELETE_LIST`に設定して、セルを実行して下さい。
```
LOCAL_USERS_DELETE_LIST = '''
student-a01@example.com
student-a02@example.com
student-a03@example.com
'''
```

In [ ]:
# 以下に、削除するローカルユーザのメールアドレスを列挙してください。
LOCAL_USERS_DELETE_LIST = '''#ここから#
# GROUP A
student-a01@example.com
student-a02@example.com
student-a03@example.com
'''#ここまで#

# ローカルユーザーの削除

In [ ]:
import os
target_hub = ['-i', os.path.expanduser('~/ansible/inventory'), 'ch-master']

!ansible -m ping {' '.join(target_hub)}

In [ ]:
import subprocess

default_user = !whoami
default_user = default_user[0]

result = subprocess.check_output(['ansible', '-m', 'ping'] + target_hub)
print(result.decode('utf8'))

In [ ]:
import re

student_mails = []
for line in LOCAL_USERS_DELETE_LIST.splitlines():
    s = re.sub(r'#.*$', '', line).strip()
    if 0 < len(s):
        student_mails.append(line)
student_mails

ユーザをDBから削除します。

In [ ]:
def get_auth_proxy():
    import subprocess
    try:
        sid = subprocess.check_output(['ansible', '-b', '-a', 'docker service ps auth-proxy -q'] + target_hub)
        sid = sid.decode('utf-8').split('\n')[1].strip()
        cinfo = subprocess.check_output(
            ['ansible', '-b', '-a', 
             'docker inspect --format "{% raw %} {{.NodeID}} {{.Status.ContainerStatus.ContainerID}} {% endraw %}" ' + sid
            ] + target_hub)
        nodeid, cid = cinfo.decode('utf-8').split('\n')[1].strip().split()
        nodeip = subprocess.check_output(
            ['ansible', '-b', '-a', 
             'docker node inspect --format "{% raw %} {{.Status.Addr}} {% endraw %}" ' + nodeid
            ] + target_hub)
        nodeip = nodeip.decode('utf-8').split('\n')[1].split()[0]
        return (nodeip, cid)
    except subprocess.CalledProcessError as e:
        print(e.output.decode('utf-8'))
        raise

auth_proxy = get_auth_proxy()
auth_proxy

In [ ]:
target_auth_proxy = ['-i', os.path.expanduser('~/ansible/inventory'), auth_proxy[0]]

local_user_infos = []
for student_mail in student_mails:
    local_user_info = subprocess.check_output(['ansible', '-b', '-a',
                                               'docker exec -i {} php /var/www/bin/del-local-user.php "{}"'.format(auth_proxy[1], student_mail)] + target_auth_proxy)
    local_user_infos.append(local_user_info.decode('utf8'))
local_user_infos[0]

講師環境の `admin_tools/data/students.tsv` に控えてあるユーザー情報を削除します。

In [ ]:
import re

for local_user_info in local_user_infos:
    local_user_info = local_user_info.split('\n')[1]
    email = local_user_info
    print(email)
    print(subprocess.check_output(['ansible', '-b', '-m', 'lineinfile', '-a',
                             'path=/jupyter/admin/admin_tools/data/students.tsv regexp="^{}.+" state=absent'.format(
                                 re.escape(email)
                             )] + target_hub).decode('utf-8'))
